# YOLO

## YOLOv11

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'  # A6000 GPU

In [3]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU capability: {torch.cuda.get_device_capability(0)}")

PyTorch version: 2.5.1+cu118
CUDA available: True
GPU count: 1
GPU name: NVIDIA RTX A6000
GPU capability: (8, 6)


In [4]:
import json
import shutil
import platform
import random
import subprocess
import sys

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from pathlib import Path
from collections import Counter, OrderedDict
from typing import Any, Dict, List, Optional, Tuple

import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms as T
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

from ultralytics import YOLO
import yaml

import ultralytics
print(ultralytics.__version__)  # 8.3.0 이상이어야 YOLOv11 지원
import yaml

8.3.243


In [5]:
# 프로젝트 루트 탐색
# 한 번만 찾고 고정
def find_project_root(marker_filename=".project-root"):
    current_dir = os.path.abspath(os.getcwd())
    while True:
        if os.path.isfile(os.path.join(current_dir, marker_filename)):
            return current_dir
        parent_dir = os.path.dirname(current_dir)
        if parent_dir == current_dir:
            raise FileNotFoundError(f"Could not find {marker_filename} in any parent directory.")
        current_dir = parent_dir
        
# 경로/출력 폴더 생성(find_project_root() 호출 후 사용)
def ensure_dir(path):
    # 디렉토리 없으면 생성
    os.makedirs(path, exist_ok=True)

PROJECT_ROOT = find_project_root()
def get_project_path(*paths):
    return os.path.join(PROJECT_ROOT, *paths)

In [8]:
# 프로젝트 경로 확인
print(f"Project Root: {PROJECT_ROOT}")

# 데이터 경로 설정
data_root = get_project_path("data/TomatOD_COCO_3")

# 데이터 구조 확인
print("\n=== 데이터 구조 ===")
for split in ['train', 'val', 'test']:
    split_path = os.path.join(data_root, split)
    img_dir = os.path.join(split_path, 'images')
    ann_file = os.path.join(split_path, 'custom_' + split + '.json')
    
    if os.path.exists(img_dir):
        img_count = len([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))])
        print(f"{split:5s}: {img_count:3d} images")

Project Root: /home/hyeonjin/tomato-detection-agentic

=== 데이터 구조 ===
train: 193 images
val  :  54 images
test :  30 images


In [7]:
# YOLO 데이터 형식 확인

def coco_to_yolo(coco_json_path, output_dir, img_width=2000, img_height=2000):
    """
    COCO JSON을 YOLO txt 형식으로 변환
    YOLO 형식: <class_id> <x_center> <y_center> <width> <height> (normalized 0~1)
    """
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # Category ID mapping (COCO → YOLO: 0-based index)
    cat_id_to_yolo = {cat['id']: idx for idx, cat in enumerate(coco_data['categories'])}
    
    # Image ID to filename mapping
    img_id_to_info = {img['id']: img for img in coco_data['images']}
    
    # Annotations grouped by image_id
    anns_by_img = {}
    for ann in coco_data['annotations']:
        img_id = ann['image_id']
        if img_id not in anns_by_img:
            anns_by_img[img_id] = []
        anns_by_img[img_id].append(ann)
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Convert each image's annotations
    for img_id, anns in anns_by_img.items():
        img_info = img_id_to_info[img_id]
        img_filename = img_info['file_name']
        txt_filename = os.path.splitext(img_filename)[0] + '.txt'
        txt_path = os.path.join(output_dir, txt_filename)
        
        # Get image dimensions
        img_w = img_info.get('width', img_width)
        img_h = img_info.get('height', img_height)
        
        with open(txt_path, 'w') as f:
            for ann in anns:
                # COCO bbox: [x, y, width, height] (absolute)
                x, y, w, h = ann['bbox']
                
                # Convert to YOLO format (normalized center coordinates)
                x_center = (x + w / 2) / img_w
                y_center = (y + h / 2) / img_h
                w_norm = w / img_w
                h_norm = h / img_h
                
                # YOLO class ID (0-based)
                class_id = cat_id_to_yolo[ann['category_id']]
                
                # Write: <class_id> <x_center> <y_center> <width> <height>
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
    
    print(f" Converted {len(anns_by_img)} annotations to YOLO format")
    return len(anns_by_img)

In [8]:
# YOLO 데이터셋 폴더 생성
yolo_dataset_root = get_project_path("data/TomatOD_YOLO_3")
ensure_dir(yolo_dataset_root)

print("=== COCO → YOLO 변환 시작 ===\n")

for split in ['train', 'val', 'test']:
    print(f"\n[{split.upper()}]")
    
    # 입력 경로
    coco_json = get_project_path(f"data/TomatOD_COCO_3/{split}/custom_{split}.json")
    src_img_dir = get_project_path(f"data/TomatOD_COCO_3/{split}/images")
    
    # 출력 경로
    dst_img_dir = os.path.join(yolo_dataset_root, split, "images")
    dst_label_dir = os.path.join(yolo_dataset_root, split, "labels")
    
    ensure_dir(dst_img_dir)
    ensure_dir(dst_label_dir)
    
    # 1. 이미지 복사
    print(f"  Copying images...")
    if os.path.exists(src_img_dir):
        for img_file in os.listdir(src_img_dir):
            if img_file.endswith(('.jpg', '.png')):
                shutil.copy(
                    os.path.join(src_img_dir, img_file),
                    os.path.join(dst_img_dir, img_file)
                )
    
    # 2. COCO → YOLO 변환
    print(f"  Converting annotations...")
    coco_to_yolo(coco_json, dst_label_dir)

print("\n✅ YOLO 데이터셋 생성 완료!")

=== COCO → YOLO 변환 시작 ===


[TRAIN]
  Copying images...
  Converting annotations...
 Converted 193 annotations to YOLO format

[VAL]
  Copying images...
  Converting annotations...
 Converted 54 annotations to YOLO format

[TEST]
  Copying images...
  Converting annotations...
 Converted 30 annotations to YOLO format

✅ YOLO 데이터셋 생성 완료!


In [7]:
from pathlib import Path

# PROJECT_ROOT를 Path 객체로 변환
project_path = Path(PROJECT_ROOT)

# 데이터 경로 설정
data_root = project_path / "data" / "TomatOD_YOLO_3"
train_dir = data_root / "train"
val_dir = data_root / "val" 
test_dir = data_root / "test"

train_images_dir = data_root / "train" / "images"
val_images_dir = data_root / "val" / "images"
test_images_dir = data_root / "test" / "images"

train_labels_dir = data_root / "train" / "labels"
val_labels_dir = data_root / "val" / "labels"
test_labels_dir = data_root / "test" / "labels"



# 클래스 정보
class_names = ["fully-ripe", "semi-ripe", "unripe"]
num_classes = len(class_names)
id2label = {i: name for i, name in enumerate(class_names)}
label2id = {name: i for i, name in enumerate(class_names)}

print(f"Classes: {class_names}")
print(f"Number of classes: {num_classes}")
print(f"Data root: {data_root}")

Classes: ['fully-ripe', 'semi-ripe', 'unripe']
Number of classes: 3
Data root: /home/hyeonjin/tomato-detection-agentic/data/TomatOD_YOLO_3


In [3]:
from urllib.request import urlretrieve
from pathlib import Path

# 다운로드할 모델 크기 선택
model_size = 'm'

# YOLOv11 모델
model_url = f"https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11{model_size}.pt"
model_path = "yolo11m.pt"

# 저장 경로
save_path = Path(f"yolo11{model_size}.pt")

if not Path(model_path).exists():
    urlretrieve(model_url, model_path)

In [ ]:
# 하이퍼파라미터
batch_size = 16
learning_rate = 0.01
num_epochs = 100
image_size = 640
confidence_threshold = 0.25
iou_threshold = 0.45

In [ ]:
# 로드
model = YOLO(model_path)

# 샘플 라벨 파일 확인
if train_labels_dir.exists():
    label_files = list(train_labels_dir.glob("*.txt"))
    if label_files:
        sample_label_file = label_files[0]
        print(f"Sample label file: {sample_label_file.name}")
        
        with open(sample_label_file, 'r') as f:
            lines = f.readlines()
            print(f"Number of objects: {len(lines)}")
            for i, line in enumerate(lines[:3]):  # 처음 3개만 출력
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id, x_center, y_center, width, height = parts[:5]
                    print(f"  Object {i+1}: class={class_id} ({class_names[int(class_id)]}), "
                          f"center=({x_center}, {y_center}), size=({width}, {height})")

# 샘플 이미지 확인
if train_images_dir.exists():
    image_files = list(train_images_dir.glob("*.jpg")) + list(train_images_dir.glob("*.png"))
    if image_files:
        sample_image_file = image_files[0]
        sample_image = Image.open(sample_image_file)
        print(f"Sample image: {sample_image_file.name}")
        print(f"Image size: {sample_image.size}")
        print(f"Image mode: {sample_image.mode}")
        

Sample label file: stereo20190406_054921_p0_snap_454.txt
Number of objects: 7
  Object 1: class=1 (semi-ripe), center=(0.661000, 0.147500), size=(0.077000, 0.077000)
  Object 2: class=2 (unripe), center=(0.365000, 0.104500), size=(0.077000, 0.077000)
  Object 3: class=0 (fully-ripe), center=(0.799500, 0.352000), size=(0.044000, 0.044000)
Sample image: stereo20190405_132743_p0_snap_488.jpg
Image size: (2000, 2000)
Image mode: RGB


In [11]:
import yaml
from pathlib import Path

# Config 파일 로드
config_path = project_path / "configs" / "yolov11" / "yolov11_3class.yaml"

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)


# Config 섹션별로 변수 할당
experiment_config = config['experiment']
model_config = config['model']
data_config = config['data']
training_config = config['training']
evaluation_config = config['evaluation']
logging_config = config['logging']
hardware_config = config['hardware']


In [12]:
# Config에서 데이터 정보 추출
class_names = data_config['class_names']
num_classes = data_config['num_classes']
data_root = project_path / data_config['data_root']

# YOLO data.yaml 생성
data_yaml_content = {
    'path': str(data_root),
    'train': 'train/images',
    'val': 'val/images', 
    'test': 'test/images',
    'nc': num_classes,
    'names': class_names
}

data_yaml_path = data_root / "data.yaml"
data_yaml_path.parent.mkdir(parents=True, exist_ok=True)

with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml_content, f, default_flow_style=False)

print(f"YOLO data.yaml created at: {data_yaml_path}")
print("Content:")
print(yaml.dump(data_yaml_content, default_flow_style=False))

YOLO data.yaml created at: /home/hyeonjin/tomato-detection-agentic/data/TomatOD_YOLO_3/data.yaml
Content:
names:
- fully-ripe
- semi-ripe
- unripe
nc: 3
path: /home/hyeonjin/tomato-detection-agentic/data/TomatOD_YOLO_3
test: test/images
train: train/images
val: val/images



In [16]:
# 학습 파라미터 설정
train_args = {
    # 데이터 설정
    'data': str(data_yaml_path),
    
    # 기본 학습 설정
    'epochs': training_config['epochs'],
    'batch': data_config['batch_size'],
    'imgsz': data_config['imgsz'],
    'device': 0,
    
    # 옵티마이저 설정
    'lr0': model_config['lr0'],
    'lrf': model_config['lrf'],
    'momentum': model_config['momentum'],
    'weight_decay': model_config['weight_decay'],
    
    # Loss 계수
    'box': model_config['box'],
    'cls': model_config['cls'],
    'dfl': model_config['dfl'],
    
    # 학습 설정
    'patience': training_config['patience'],
    'save': training_config['save'],
    'save_period': training_config['save_period'],
    'cache': training_config['cache'],
    'val': training_config['val'],
    'project': training_config['project'],
    'name': training_config['name'],
    'exist_ok': training_config['exist_ok'],
    'plots': training_config['plots'],
    'amp': training_config['amp'],
    'fraction': training_config['fraction'],
    'profile': training_config['profile'],
    'freeze': training_config['freeze'],
    'multi_scale': training_config['multi_scale'],
    'overlap_mask': training_config['overlap_mask'],
    'mask_ratio': training_config['mask_ratio'],
    'dropout': training_config['dropout'],
    
    
    # 데이터 증강 설정
    'hsv_h': data_config['augmentation']['hsv_h'],
    'hsv_s': data_config['augmentation']['hsv_s'],
    'hsv_v': data_config['augmentation']['hsv_v'],
    'degrees': data_config['augmentation']['degrees'],
    'translate': data_config['augmentation']['translate'],
    'scale': data_config['augmentation']['scale'],
    'shear': data_config['augmentation']['shear'],
    'perspective': data_config['augmentation']['perspective'],
    'flipud': data_config['augmentation']['flipud'],
    'fliplr': data_config['augmentation']['fliplr'],
    'mosaic': data_config['augmentation']['mosaic'],
    'mixup': data_config['augmentation']['mixup'],
    
    # 기타 설정 (hardware 섹션에서 가져오기)
    'seed': config['experiment']['seed'],
    'deterministic': config['hardware']['deterministic'],
    'cos_lr': config['hardware']['cos_lr'],  # training_config가 아닌 hardware에서 가져오기
    'close_mosaic': config['hardware']['close_mosaic']  # training_config가 아닌 hardware에서 가져오기
}


In [14]:
# Config에서 모델 설정 추출
model_config = config['model']
model_name = model_config['pretrained_path']

# YOLOv11 모델 로드
model = YOLO(model_name)
print(f"YOLOv11 model loaded: {model_name}")
print(f"Model size: {model_config['model_size']}")

# 모델 정보 출력
try:
    model_info = model.info(verbose=False)
    print(f"Model parameters: {model_info}")
except:
    print("Model info not available")

YOLOv11 model loaded: yolo11m.pt
Model size: m
Model parameters: None


In [ ]:
# 학습 실행
print("=" * 60)
print("YOLO TRAINING STARTED")
print("=" * 60)
print(f"Experiment: {config['experiment']['name']}")
print(f"Model: {model_config['arch_name']} ({model_config['model_size']})")
print(f"Dataset: {data_config['dataset_name']}")
print(f"Epochs: {training_config['epochs']}")
print(f"Batch size: {data_config['batch_size']}")
print(f"Device: {train_args['device']}")
print("=" * 60)

# 학습 시작
results = model.train(**train_args)

print("\nTraining completed!")
print(f"Results: {results}")

YOLO TRAINING STARTED
Experiment: yolo11_tomato_3class_exp001
Model: yolov11 (m)
Dataset: TomatOD_YOLO_3
Epochs: 50
Batch size: 8
Device: 0
New https://pypi.org/project/ultralytics/8.3.246 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.243 🚀 Python-3.12.3 torch-2.5.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48550MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/hyeonjin/tomato-detection-agentic/data/TomatOD_YOLO_3/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_widt

: 

In [17]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU count: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU capability: {torch.cuda.get_device_capability(0)}")

PyTorch version: 2.5.1+cu118
CUDA available: True
CUDA version: 11.8
GPU count: 1
GPU name: NVIDIA RTX A6000
GPU capability: (8, 6)


In [70]:
print("=" * 60)
print("EXPERIMENTAL ENVIRONMENT")
print("=" * 60)

# 하드웨어 정보
print("\n[Hardware]")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"  Device: {device}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"  GPU: {gpu_name}")
    print(f"  GPU Memory: {gpu_memory:.1f} GB")
    print(f"  CUDA Version: {torch.version.cuda}")
    if torch.backends.cudnn.is_available():
        print(f"  cuDNN Version: {torch.backends.cudnn.version()}")


EXPERIMENTAL ENVIRONMENT

[Hardware]
  Device: cuda
  GPU: NVIDIA RTX PRO 6000 Blackwell Max-Q Workstation Edition
  GPU Memory: 95.0 GB
  CUDA Version: 11.8
  cuDNN Version: 90100


# YOLO v12

In [1]:
import urllib.request
from pathlib import Path

# 다운로드할 모델 크기 선택 (n, s, m, l, x)
model_size = 'm'  # nano, small, medium, large, xlarge

# YOLOv12 모델 URL (Ultralytics 공식)
model_url = f"https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo12{model_size}.pt"

# 저장 경로
save_path = Path(f"yolo12{model_size}.pt")

# 다운로드
print(f"Downloading YOLOv12-{model_size} from {model_url}...")
urllib.request.urlretrieve(model_url, save_path)

(PosixPath('yolo12m.pt'), <http.client.HTTPMessage at 0x7ff13f654890>)

In [8]:
import yaml

# 데이터 경로 설정
data_root = get_project_path("data/TomatOD_YOLO_3")

project_path = Path(PROJECT_ROOT)
data_root = project_path / "data" / "TomatOD_YOLO_3"
train_dir = data_root / "train"
val_dir = data_root / "val" 
test_dir = data_root / "test"

train_images_dir = data_root / "train" / "images"
val_images_dir = data_root / "val" / "images"
test_images_dir = data_root / "test" / "images"

train_labels_dir = data_root / "train" / "labels"
val_labels_dir = data_root / "val" / "labels"
test_labels_dir = data_root / "test" / "labels"

data_yaml_path = data_root / "data.yaml"

# 클래스 정보
class_names = ["fully-ripe", "semi-ripe", "unripe"]
num_classes = len(class_names)
id2label = {i: name for i, name in enumerate(class_names)}
label2id = {name: i for i, name in enumerate(class_names)}

print(f"Classes: {class_names}")
print(f"Number of classes: {num_classes}")
print(f"data.yaml created at: {data_yaml_path}")

Classes: ['fully-ripe', 'semi-ripe', 'unripe']
Number of classes: 3
data.yaml created at: /home/hyeonjin/tomato-detection-agentic/data/TomatOD_YOLO_3/data.yaml


In [11]:
# config 파일 로드
import yaml
from pathlib import Path
from ultralytics import YOLO

project_path = Path(PROJECT_ROOT)
config_path = project_path / "configs" / "yolov12" / "yolov12_3class.yaml"
print(f"Config path: {config_path}")

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

Config path: /home/hyeonjin/tomato-detection-agentic/configs/yolov12/yolov12_3class.yaml


In [12]:
# 학습 파라미터 설정 및 모델 파라미터 추출 
model_config = config['model']
data_config = config['data']
training_config = config['training']

pretrained_path = model_config['pretrained_path']
model_size = model_config['model_size']
num_classes = model_config['num_classes']
class_names = data_config['class_names']

In [15]:
model = YOLO(pretrained_path)

print(f"Model type: {type(model)}")
print(f"Model info: {model.info()}")

Model type: <class 'ultralytics.models.yolo.model.YOLO'>
YOLOv12m summary: 292 layers, 20,201,216 parameters, 0 gradients, 68.1 GFLOPs
Model info: (292, 20201216, 0, 68.0792576)


In [13]:
train_args = {
    # 데이터 설정
    'data': str(data_yaml_path),
    
    # 기본 학습 설정
    'epochs': training_config['epochs'],
    'batch': data_config['batch_size'],
    'imgsz': data_config['imgsz'],
    'device': training_config['device'],
    
    # 옵티마이저 설정
    'lr0': model_config['lr0'],
    'lrf': model_config['lrf'],
    'momentum': model_config['momentum'],
    'weight_decay': model_config['weight_decay'],
    
    # Loss 계수
    'box': model_config['box'],
    'cls': model_config['cls'],
    'dfl': model_config['dfl'],
    
    # 학습 설정
    'patience': training_config['patience'],
    'save': training_config['save'],
    'save_period': training_config['save_period'],
    'cache': training_config['cache'],
    'val': training_config['val'],
    'project': training_config['project'],
    'exist_ok': training_config['exist_ok'],
    'plots': training_config['plots'],
    'amp': training_config['amp'],
    'fraction': training_config['fraction'],
    'seed': config['experiment']['seed'],
    
    # 데이터 증강 설정
    'hsv_h': data_config['augmentation']['hsv_h'],
    'hsv_s': data_config['augmentation']['hsv_s'],
    'hsv_v': data_config['augmentation']['hsv_v'],
    'degrees': data_config['augmentation']['degrees'],
    'translate': data_config['augmentation']['translate'],
    'scale': data_config['augmentation']['scale'],
    'shear': data_config['augmentation']['shear'],
    'perspective': data_config['augmentation']['perspective'],
    'flipud': data_config['augmentation']['flipud'],
    'fliplr': data_config['augmentation']['fliplr'],
    'mosaic': data_config['augmentation']['mosaic'],
    'mixup': data_config['augmentation']['mixup'],
    
    # 기타 설정
    'deterministic': config['hardware']['deterministic'],
    'cos_lr': config['hardware']['cos_lr'],
    'close_mosaic': config['hardware']['close_mosaic'],
}

# name이 None이 아닌 경우에만 추가
if training_config.get('name') is not None:
    train_args['name'] = training_config['name']

# None 값 제거
train_args = {k: v for k, v in train_args.items() if v is not None}

# TensorBoard는 환경변수로 활성화
import os
os.environ['TENSORBOARD'] = '1'

In [16]:
# 학습
result = model.train(**train_args)

New https://pypi.org/project/ultralytics/8.3.248 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.243 🚀 Python-3.12.3 torch-2.5.1+cu118 CUDA:2 (NVIDIA RTX A6000, 48550MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/hyeonjin/tomato-detection-agentic/data/TomatOD_YOLO_3/data.yaml, degrees=0.0, deterministic=True, device=2, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=F

In [ ]:
# 가중치 파일 확인
project_path = Path(PROJECT_ROOT)
weights_dir = project_path / "notebook" / "exp" / "yolo" / "train5"

if weights_dir.exists():
    best_pt = weights_dir/ "weights" / "best.pt"
    last_pt = weights_dir / "weights" / "last.pt"
    
    print(f"\nModel weights:")
    if best_pt.exists():
        print(f"  Best model: {best_pt}")
        print(f"  Size: {best_pt.stat().st_size / 1024 / 1024:.2f} MB")
    if last_pt.exists():
        print(f"  Last model: {last_pt}")
        print(f"  Size: {last_pt.stat().st_size / 1024 / 1024:.2f} MB")

# 결과 파일 확인
results_files = list(Path(results.save_dir).glob("*.csv")) + \
                list(Path(results.save_dir).glob("*.png")) + \
                list(Path(results.save_dir).glob("*.jpg"))
print(f"\nResult files ({len(results_files)}):")
for f in results_files[:10]:  # 처음 10개만 출력
    print(f"  {f.name}")

print("\n" + "=" * 60)
print("Finetuning Completed!")
print("=" * 60)


Model weights:
  Best model: /home/hyeonjin/tomato-detection-agentic/notebook/exp/yolo/train5/weights/best.pt
  Size: 38.89 MB
  Last model: /home/hyeonjin/tomato-detection-agentic/notebook/exp/yolo/train5/weights/last.pt
  Size: 38.89 MB


NameError: name 'results' is not defined